In [ ]:
!pip install gspread oauth2client openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('[CREDENTIALS_FILE]', scope)
client = gspread.authorize(credentials)

sheet = client.open("[GOOGLE_SHEET_NAME]").sheet1
keywords = sheet.col_values(1) # Assuming the keywords are in column A

In [ ]:
## OPENAI API KEY BELOW

import openai
openai.api_key = '[OPENAI_API_KEY]'

## OPENAI API KEY ABOVE

In [ ]:
from googleapiclient.discovery import build
import openai

# Set up the Google Docs API client
docs_service = build('docs', 'v1', credentials=credentials)
# Set up the Google Drive API client
drive_service = build('drive', 'v3', credentials=credentials)

data = sheet.get_all_values()

# Separate into two lists
keywords = [row[0] for row in data]
status = [row[1] if len(row) > 1 else '' for row in data]
# intents = [row[2] if len(row) > 2 else '' for row in data]
# styles = [row[3] if len(row) > 3 else '' for row in data]

for keyword, mark in zip(keywords, status):
    # Check if the keyword is not processed yet (mark would be empty)
    if keyword.strip() and not mark:
        print(f"Processing keyword {keyword}...")
        # Rest of your code to handle the keyword
    else:
        print(f"Skipping empty keyword or already marked keyword: {keyword}")

    if not mark:
        # Generate the content using OpenAI API
        content = openai.Completion.create(
          engine="text-davinci-003",
          prompt=f"I need an outline for a professional 6-7 section blog for the keyword {keyword}. This blog article will be written with targeted precision for this keyword, and to meet Google's E-A-T standards. It is imperative the outline should be formatted as follows: [Roman Numeral](Section Title) [A.](bulletpoint 1) [B.](bulletpoint 2) [C.](bulletpoint 3)",
          max_tokens=3980
        )

        # Create a new Google Document
        document = docs_service.documents().create().execute()
        # Document ID of the created Google Document
        document_id = document['documentId']

        # Write the content to the Google Document
        requests = [
            {
                'insertText': {
                    'location': {
                        'index': 1,
                    },
                    'text': content.choices[0].text
                }
            }
        ]
        docs_service.documents().batchUpdate(documentId=document['documentId'], body={'requests': requests}).execute()

        print(f"Document created for keyword {keyword} with Document ID: {document['documentId']}")

        # Mark the keyword as processed in the Google Sheet by writing the Document ID
        sheet.update_cell(keywords.index(keyword) + 1, 2, document['documentId'])

        # Define the permissions to add
        permissions = {
            'role': 'writer',
            'type': 'anyone'
        }

        # Update the permissions for the document
        drive_service.permissions().create(
            fileId=document_id,
            body=permissions
        ).execute()


In [ ]:
import re
import openai

from googleapiclient.discovery import build

spreadsheet = client.open('[GOOGLE_SHEET_NAME]')

# Get the spreadsheet ID
spreadsheet_id = spreadsheet.id

# Build services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

# Now, use the googleapiclient's Sheets API to get the data from column B
sheet_service = build('sheets', 'v4', credentials=credentials)
range_ = 'Sheet1!B:B' # Adjust this to your specific sheet and column
result = sheet_service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_).execute()
document_ids = result.get('values', [])

# Get rows from columns A to C
result = sheet_service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range='Sheet1!A1:C').execute()
values = result.get('values', [])

# Process each row
for row_number, row in enumerate(values, start=1):
    if len(row) >= 2:
        document_id = row[1]
        print(f"Processing document {row_number}: {document_id}")
        if document_id and (len(row) < 3 or not row[2]):
            document = docs_service.documents().get(documentId=document_id).execute()
            paragraphs = [element['paragraph'] for element in document['body']['content'] if 'paragraph' in element]
            paragraph_texts = [paragraph['elements'][0]['textRun']['content'] for paragraph in paragraphs if paragraph['elements']]

            sections = []
            current_section = ""
            for text in paragraph_texts:
                if re.match(r'^[IVX]+\.', text.strip()):
                    if current_section:
                        sections.append(current_section)
                    current_section = text
                    print(f"Starting new section with text: {text}")
                else:
                    current_section += text
                    print(f"Appending to current section: {text}")
            if current_section:
                sections.append(current_section)

            sections = [section.strip('\n') for section in sections]
            openai_api_responses = []
            for paragraph in sections:
                if paragraph.strip():
                    print(f"Section: {paragraph}")
                    response = openai.ChatCompletion.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "system", "content": "You are a pulitzer prize winning [PUT YOUR NICHE HERE] writer"},
                            {"role": "user", "content": f"I will give you one section of an outline, The Title (which is a Roman Numeral), and lettered bullet points. I need you to write the Roman Numeral Title at the top, and write about 350 words about the lettered bullet points provided. Please write in an style that is professional but also excited, making sure to generate intrigue about a groundbreaking technology (AI) and how business owners can apply it to their business. Make it meet Google's E-A-T standards. You must take note that this section is just a piece of a whole blog article, not the entire whole article, so consider that in your writing. Please only use 100% unique human-written content. Here is the outline section: {paragraph}"},
                        ]
                    )
                    print(f"API response for paragraph '{paragraph[:50]}...': {response}")
                    response_text = response['choices'][0]['message']['content']
                    openai_api_responses.append(response_text + "\n\n")

            openai_api_responses = openai_api_responses[::-1]
            new_document = docs_service.documents().create().execute()
            new_document_id = new_document['documentId']

            for response in openai_api_responses:
                if response:
                    docs_service.documents().batchUpdate(
                        documentId=new_document_id,
                        body={
                            'requests': [
                                {'insertText': {
                                    'location': {
                                        'index': 1,
                                    },
                                    'text': response
                                }}
                            ]
                        }
                    ).execute()
                else:
                    print("Skipping empty response")
            print("Document ID: ", new_document_id)

            update_range = f'Sheet1!C{row_number}' # Update the row dynamically
            sheet_service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=update_range,
                valueInputOption="RAW",
                body={
                    "values": [[new_document_id]]
                }
            ).execute()

            # Define the permissions to add
            permissions = {
                'role': 'writer',
                'type': 'anyone'
            }

            # Update the permissions for the document
            drive_service.permissions().create(
                fileId=new_document_id,
                body=permissions
            ).execute()